<a href="https://colab.research.google.com/github/PaoloComensoli/CompitinoDataViz/blob/main/first_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
# Setup libraries and data
import pandas as pd
import numpy as np
import altair as alt
proms = pd.read_csv('https://raw.githubusercontent.com/PaoloComensoli/CompitinoDataViz/main/compitino_dataviz_2021_2022.csv')

In [43]:
score_gain = proms['score_postop'] - proms['score_preop']
proms['score_gain'] = score_gain
score_gain_category = []
score_gain_category_index = []
for delta in proms['score_gain']:
  if delta < -5 :
    score_gain_category.append('Worsening')
  elif -5 <= delta <= 5:
    score_gain_category.append('Stable')
  else:
    score_gain_category.append('Improvement')
proms['score_gain_category'] = score_gain_category

unusualCondtitions = []

englishWeathers = {}
englishWeathers['poco nuvoloso'] = 'Partly Cloudy'
englishWeathers['nebbia al mattino'] = 'Morning Fog'
englishWeathers['sereno'] = 'Clear'
englishWeathers['pioggia e schiarite'] = 'Rain and Clearing'
englishWeathers['pioggia debole'] = 'Weak Rain'
englishWeathers['nubi sparse'] = 'Scattered Clouds'
englishWeathers['temporale e schiarite'] = 'Thunderstorm and Clearing'
englishWeathers['neve'] = 'Snow'
englishWeathers['coperto'] = 'Covered'
englishWeathers['neve e schiarite'] = 'Snow and Clearing'
englishWeathers['neve debole'] = 'Light Snow'
englishWeathers['sole e caldo'] = 'Sun and Hot'
englishWeathers['pioggia mista a neve e schiarite'] = 'Mixed rain with snow and Clearing'
englishWeathers['nebbia'] = 'Fog'
englishWeathers['pioggia'] = 'Rain'

proms.replace({"condizioni_meteo" : englishWeathers}, inplace=True)

for cond in proms['condizioni_meteo'].drop_duplicates():
  condResults = proms[proms['condizioni_meteo'] == cond]
  if len(condResults) < 50:
    unusualCondtitions.append(cond)
    proms.loc[proms['condizioni_meteo'] == cond, 'condizioni_meteo'] = "*Unusual conditions"

conditions = list(proms['condizioni_meteo'].drop_duplicates())
modulistiche = ["SF12 PhysicalScore", "SF12 MentalScore"]

stable = []
worse = []
improve = []
conditionsWithNumbers = []
customIndex = 0
condTranslation = {}
dConditionWorseMental = {}

for cond in conditions: 
  condResults = proms[proms['condizioni_meteo'] == cond]
  worseTable = condResults[condResults['score_gain_category'] == 'Worsening']
  stableTable = condResults[condResults['score_gain_category'] == 'Stable']
  improveTable = condResults[condResults['score_gain_category'] == 'Improvement']
  value = len(worseTable[worseTable['nome_modulistica'] == 'SF12 MentalScore']) / (len(worseTable[worseTable['nome_modulistica'] == 'SF12 MentalScore']) +len(stableTable[stableTable['nome_modulistica'] == 'SF12 MentalScore']) +len(improveTable[improveTable['nome_modulistica'] == 'SF12 MentalScore']))
  dConditionWorseMental[cond] = value

def my_sort_tuple(tup): 
    return(sorted(tup, key = lambda x: x[1])) 
condOrdered = my_sort_tuple(dConditionWorseMental.items())[::-1]

for cond in [t[0] for t in condOrdered]:
  cLen = int(len(proms[proms['condizioni_meteo'] == cond]) / 2)
  newName = f'{customIndex}       {cond} ({cLen}x2)'
  condTranslation[cond] = newName
  conditionsWithNumbers.append(newName)
  customIndex += 1

proms.replace({"condizioni_meteo" : condTranslation}, inplace=True)

for cond in conditionsWithNumbers:
  condResults = proms[proms['condizioni_meteo'] == cond]
  worseTable = condResults[condResults['score_gain_category'] == 'Worsening']
  stableTable = condResults[condResults['score_gain_category'] == 'Stable']
  improveTable = condResults[condResults['score_gain_category'] == 'Improvement']
  worse.append([len(worseTable[worseTable['nome_modulistica'] == 'SF12 PhysicalScore']), len(worseTable[worseTable['nome_modulistica'] == 'SF12 MentalScore'])])
  stable.append([len(stableTable[stableTable['nome_modulistica'] == 'SF12 PhysicalScore']), len(stableTable[stableTable['nome_modulistica'] == 'SF12 MentalScore'])])
  improve.append([len(improveTable[improveTable['nome_modulistica'] == 'SF12 PhysicalScore']), len(improveTable[improveTable['nome_modulistica'] == 'SF12 MentalScore'])])
  

worseResults=pd.DataFrame(worse,index=conditionsWithNumbers,columns=modulistiche)
stableResults=pd.DataFrame(stable,index=conditionsWithNumbers,columns=modulistiche)
improveResults=pd.DataFrame(improve,index=conditionsWithNumbers,columns=modulistiche)

def prep_df(df, name):
    df = df.stack().reset_index()
    df.columns = ['c1', 'c2', 'values']
    df['Category'] = name
    return df

N_p = worseResults['SF12 PhysicalScore'] + stableResults['SF12 PhysicalScore'] + improveResults['SF12 PhysicalScore']
N_m = worseResults['SF12 MentalScore'] + stableResults['SF12 MentalScore'] + improveResults['SF12 MentalScore']

worseResults['Physical Score'] = (worseResults['SF12 PhysicalScore']/N_p)
worseResults['Mental Score'] = (worseResults['SF12 MentalScore']/N_m)
worseResults = worseResults.drop(['SF12 PhysicalScore','SF12 MentalScore'],axis= 1)

stableResults['Physical Score'] = (stableResults['SF12 PhysicalScore']/N_p)
stableResults['Mental Score'] = (stableResults['SF12 MentalScore']/N_m)
stableResults = stableResults.drop(['SF12 PhysicalScore','SF12 MentalScore'],axis= 1)

improveResults['Physical Score'] = (improveResults['SF12 PhysicalScore']/N_p)
improveResults['Mental Score'] = (improveResults['SF12 MentalScore']/N_m)
improveResults = improveResults.drop(['SF12 PhysicalScore','SF12 MentalScore'],axis= 1)

worseResults = prep_df(worseResults, 'Worsening (Δ < -5)')
stableResults = prep_df(stableResults, 'Stable (-5 < Δ < 5)')
improveResults = prep_df(improveResults, 'Improving (Δ > 5)')

df = pd.concat([worseResults, stableResults, improveResults])

In [44]:
plot_title = alt.TitleParams("Will weather affect operations perceived outcome?", subtitle=["Comparison in the ratio of the improvements, worsening and stability (Physical and Mental) of patients following a medical operation in the various weather conditions"])

barStacked = alt.Chart(df, title=plot_title).mark_bar(
    cornerRadiusTopLeft=2,
    cornerRadiusTopRight=2
).encode(
     x=alt.Y('sum(values):Q',
        axis=alt.Axis(
            grid=False,
            format='%',
            title='Results rate')),
    y=alt.X('c2:N', title=None),
    row=alt.Row('c1:N', title='Weather conditions',header=alt.Header(labelAngle=0,labelAlign='left')),
    color=alt.Color('Category:N',
            scale=alt.Scale(
                range=['#96ceb4','#ffcc5c','#ff6f69'],
            ),
        )
).properties(width=1700)
alt.concat(barStacked,
    title=alt.TitleParams(
        f'*Unusual conditions (# < 50) = {str(unusualCondtitions)[1:-1]}',
        baseline='bottom',
        orient='bottom',
        offset=30,
        anchor='start'
    )
)

alt.ConcatChart(...)

# Aggiungiamo alcune rifiniture grafiche
<img src="https://github.com/PaoloComensoli/CompitinoDataViz/blob/main/first_visualization_2021.png?raw=true"/>